# Implementing the fun filters from AA228 on my own!
1. Extended Kalman Filter
2. Unscented Kalman Filter
3. Particle Filter (not right now, but maybee in the future?)

With scenarios to generate data for the filters
   

In [11]:
import matplotlib.pyplot as plt
import numpy as np
import sys
from pprint import pprint
sys.path.append("../..")
from estimation.filtering.ekf import State, EkfParams, EKF

# from importlib import reload  # Python 3.4+



In [15]:
# State
example_state = State(1,np.diag([1,1,1]))
print(example_state)

State(x=1, P=array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 1]]))


# Scenarios

## AA272 (GPS Class) HW4

State
$$
\mu_t := \begin{bmatrix} x_{t} & y_{t} & \theta_t \end{bmatrix}^\top,
$$

Input
$$
u_t := \begin{bmatrix} v_t & \alpha_t \end{bmatrix}^\top
$$


### Setup

State transition
$$
\begin{aligned}
\mu_t &= f(\mu_{t-1}, u_{t}) + w_t, \\
&= \underbrace{\mu_{t-1} + \Delta t \begin{bmatrix} v_{t} \cos(\theta_{t-1}) \\ v_{t} \sin(\theta_{t-1}) \\ \alpha_{t} \end{bmatrix}}_{f(\mu_{t-1}, u_{t})} + w_t,
\end{aligned}
$$

In [ ]:
def state_transition_model(dt, state, u):
    """
    Nominal state transition model
        Inputs:
            dt: timestep [float]
            state: current state (3,) [x, y, theta]
            u_cur: current control input (2,) [v, alpha]
        Outputs:
            mu_next: next state (3,) [x, y, theta]
    """
    _,_,theta = state
    v,alpha = u
    return (state + dt * np.ndarray(
        [v*np.cov(theta), v*np.sin(theta), alpha]
    ))

def simulate_noisy_state_transition(state, u, Q):
    """
    Simulate noisy state transition
    Inputs:
        state: current state mean (3,) [x, y, theta]
        u: current control input (2,) [v, alpha]
        Q: process noise covariance (3x3)
    Outputs:
        state_next: next state mean (3x1) [v, alpha]
    """
    state_next = state_transition_model(state, u) + np.random.multivariate_normal(np.zeros(3), Q)
    return state_next



Measurements

$$
h^{(i)}(\mu_t) = \sqrt{(x^{(i)} - x_t)^2 + (y^{(i)} - y_t)^2},
$$

In [26]:
def measurement_model(state):
    """
    Measurement model
        Inputs:
            state: current state mean (3,) [x, y, theta]
        Outputs:
            z: measurement vector (4,) [d_1, d_2, d_3, d_4]
    """
    # Absolute beacon positions
    posb = 120
    xy_beacon = np.array([[-posb, posb], [posb, posb], [posb, -posb], [-posb, -posb]])
    
    z = np.zeros(4)
    x,y,_ = state

    for i, b in enumerate(xy_beacon):
        xb,yb = b
        z[i] = np.sqrt( (x - xb)**2 + (y - yb)**2 )

        return z

# simulate noisy meas
def simulate_noisy_measurement(state, R):
    """
    Simulate noisy measurement
        Inputs:
        state: current state mean (3,) [x, y, theta]
        R: measurement noise covariance (4x4)
        Outputs:
            z_meas: measurement vector (4,) [d_1, d_2, d_3, d_4]
    """
    z_meas = measurement_model(state) + np.random.multivariate_normal(np.zeros(4), R)
    return z_meas

State transition and Measurement covariances
$$
\begin{aligned}
F(\mu_t, u_t) &= \begin{bmatrix} 1 & 0 & -\Delta t\ v_t sin(\theta_{t-1}) \\
    0 & 1 & \Delta t\ v_t cos(\theta_{t-1}) \\
    0 & 0 & 1 \end{bmatrix}
\end{aligned}
$$

$$
\begin{aligned}
H_t &= \begin{bmatrix}
\frac{2(x^1 - x_t)(-1)}{2 \sqrt{(x^1 - x_t)^2 + (y^1 - y_t)^2}} &
\frac{2(y^1 - y_t)(-1)}{2 \sqrt{(x^1 - x_t)^2 + (y^1 - y_t)^2}} &
0 \\
& \vdots & \\
\frac{2(x^4 - x_t)(-1)}{2 \sqrt{(x^4 - x_t)^2 + (y^4 - y_t)^2}} &
\frac{2(y^4 - y_t)(-1)}{2 \sqrt{(x^4 - x_t)^2 + (y^4 - y_t)^2}} &
0 \\
\end{bmatrix} \\ \\
&= \begin{bmatrix}
\frac{-(x^1 - x_t)}{\sqrt{(x^1 - x_t)^2 + (y^1 - y_t)^2}} &
\frac{-(y^1 - y_t)}{\sqrt{(x^1 - x_t)^2 + (y^1 - y_t)^2}} &
0 \\
& \vdots & \\
\frac{-(x^4 - x_t)}{\sqrt{(x^4 - x_t)^2 + (y^4 - y_t)^2}} &
\frac{-(y^4 - y_t)}{\sqrt{(x^4 - x_t)^2 + (y^4 - y_t)^2}} &
0 \\
\end{bmatrix} \\
\end{aligned}
$$

In [ ]:
def get_F(dt, state, u):
    """
    Compute the Jacobian of the state transition model
    Inputs:
        dt: timestep [float]
        state: current state mean (3,) [x, y, theta]
        u: current control input (2,) [v, alpha]
    Outputs:
        F: Jacobian matrix (3x3)
    """
    _,_,theta = state
    v,_ = u
    F = np.array([
      [1, 0, -dt * v * np.sin(theta)],
      [0, 1, dt * v * np.cos(theta)],
      [0, 0, 1]
    ])

    return F

def get_H(state):
    """
    Compute the Jacobian of the measurement model
    Inputs:
        state: current state mean (3,) [s_t, alpha, theta]
    Outputs:
        H: Jacobian matrix (4x3)
    """

    x,y,_  = state
    H = np.zeros((4,3))


    # distances from beacons
    xy_beacon = np.array([[-120, 120], [120, 120], [120, -120], [-120, -120]])

    # Did this the long way at first
    # d1 = np.sqrt((x - xy_beacon[0][0])**2 + (y - xy_beacon[0][1])**2)
    # d2 = np.sqrt((x - xy_beacon[1][0])**2 + (y - xy_beacon[1][1])**2)
    # d3 = np.sqrt((x - xy_beacon[2][0])**2 + (y - xy_beacon[2][1])**2)
    # d4 = np.sqrt((x - xy_beacon[3][0])**2 + (y - xy_beacon[3][1])**2)


    # H = np.array([
    #     [(x - xy_beacon[0][0]) / d1, (y - xy_beacon[0][1]) / d1, 0],
    #     [(x - xy_beacon[1][0]) / d2, (y - xy_beacon[1][1]) / d2, 0],
    #     [(x - xy_beacon[2][0]) / d3, (y - xy_beacon[2][1]) / d3, 0],
    #     [(x - xy_beacon[3][0]) / d4, (y - xy_beacon[3][1]) / d4, 0],
    # ])

    x_dist = x - xy_beacon[:,0]
    y_dist = x - xy_beacon[:,1]
    distances = np.sqrt(
        x_dist**2 + y_dist**2
    )

    H = np.vstack(
        ((x_dist / distances), (y_dist / distances), np.zeros(4))
    ).T

    return H

state = np.array([1,1,1])
get_H(state)

[[ 0.71297458 -0.70118988  0.        ]
 [-0.70710678 -0.70710678  0.        ]
 [-0.70118988  0.71297458  0.        ]
 [ 0.70710678  0.70710678  0.        ]]
[ 0.71297458 -0.70710678 -0.70118988  0.70710678]
[[ True  True  True]
 [ True  True  True]
 [ True  True  True]
 [ True  True  True]]


array([[ 0.71297458, -0.70118988,  0.        ],
       [-0.70710678, -0.70710678,  0.        ],
       [-0.70118988,  0.71297458,  0.        ],
       [ 0.70710678,  0.70710678,  0.        ]])

In [ ]:
# Scenario from the AA272 (GPS) HW4 where 

def setup_gps_environment():
    delta_t = 1  # time step
    Q = np.diag(np.array([0.1, 0.1, 0.01]))  # process noise covariance
    R = 10 * np.eye(4)  # measurement noise covariance
    seed = 0   # random seed for reproducibility (do not change!!!)
    lent = 100 # length of trajectory
    n_mc = 4   # number of Monte Carlo simulations
    mu0 = np.array([0, 0, 0])  # initial state [s_0, alpha, theta]

    # control input
    st = 5   # speed
    alpha = 0.1  # angular rate
    u_cur = np.array([st, alpha]) # current control input (2x1) [s_t, alpha]

    # set random seed
    np.random.seed(seed)

    env = {
        'delta_t': delta_t,
        'Q': Q,
        'R': R,
        'lent': lent,
        'n_mc': n_mc,
        'mu0': mu0,
        'u_cur': u_cur,
        'seed': seed
    }

    return env
